# Homework 1

* Peyman Naseri, `96100522`
* Mobina Pournemat, `97105833`
* Mahsa Amani, `97105769`

# Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


link to folder: https://drive.google.com/drive/folders/1evTdWmxf1lCCKtz7uAQWuQbUoId64nKd?usp=sharing

In [ ]:
# or where you put the folder
%cd /content/drive/MyDrive/NLP/HW1/

/content/drive/MyDrive/NLP/HW1


In [ ]:
! ls

farsi_names.sql  places.txt			  resources
parsi_io	 QA_exatraction_with_RegEx.ipynb  verb_to_bon.json


<div dir=rtl>
با توجه به این که تعدادی فایل شامل فایل اسامی فارسی، فایل مکان ها، بن فعل ها و نیز tagger موجود در کتابخانه hazm نیاز داریم، آن ها را از فولدر درایو می خوانیم.
</div>

# Install and import required packages


In [ ]:
! pip install hazm
! pip install parstdex

     |████████████████████████████████| 316 kB 23.5 MB/s 
     |████████████████████████████████| 233 kB 46.8 MB/s 
     |████████████████████████████████| 1.4 MB 47.6 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=51498f2e911e17cde48b319adb4327ca2278377c330df17ccca08beb65704613
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=155216 sha256=a1142e066e6de06abfc4df43b9672e515df0284a88645a843989dce83927eb1a
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 42 kB 964 kB/s 
     |████████████████████████████████| 280 kB 35.9 MB/s 
     |

In [ ]:
# !pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade

In [ ]:
import re
from __future__ import unicode_literals
from hazm import *
import json
from parstdex import Parstdex
from parsi_io.modules.number_extractor import NumberExtractor

<div dir=rtl>
ابتدا تمام نیازمندی ها را نصب و ایمپورت می کنیم.
</div>

# Question & Answer Extraction

In [ ]:
class TimeExtraction(object):
    def __init__(self):
        self.model = Parstdex()

    def run(self, text):
        result = self.model.extract_marker(text)
        return result

In [ ]:
class QAExtraction:
    def __init__(self):
        super().__init__()
        self.load_files()
        self.tagger = POSTagger(model='resources/postagger.model')
        self.live_patterns = self.names + ["من", "تو", "او", "ما", "شما", "آن ها", "آنها", "دوست", "رفیق", "همسایه", "همکار", "دشمن"]

    def load_files(self):
        # file to find root of a verb
        with open('verb_to_bon.json', encoding='utf-8') as f:
            self.v_to_thirdperson = json.load(f)

        # file containing persian names
        with open('farsi_names.sql') as f:
            self.names = [eval(line.rstrip('\n').split(",")[1]) for line in f]

        # file containing name of places/locations
        with open('places.txt') as f:
            self.places = f.read().split("|")

    def check_live_noun(self, text):
        text_arr = text.split()
        for t in text_arr:
            if t in self.live_patterns:
                return True
        return False
    
    def check_place(self, text):
        if text in self.places:
            return True
        return False

    def transitive_verbs(self, text):
        text = text.replace("\u200c", " ")
        result = []
        regexes = re.finditer(r"\sرا\s", text)
        prev_e = 0
        for regex in regexes:
            s, e = regex.start(), regex.end()
            text_arr = text.split()
            index_of_ra = text_arr.index("را", prev_e)
            prev_e = index_of_ra + 1
            A = text_arr[index_of_ra-1]
            # check live and not live subject
            live_or_not = self.check_live_noun(A)
            if live_or_not:
                live_vs_not = " چه کسی را "
            else:
                live_vs_not = " چه چیزی/کسی را "
            Q = " ".join(text_arr[:index_of_ra-1]) + live_vs_not + text[e:-1] + "؟"  
            result.append({"Question": Q, "Answer": A})
        return result


    def esnadi_verbs(self, text):
        text = text.replace("\u200c", " ")
        result = []
        regexes = re.finditer(r"(است|بود|شد|گشت|گردید|هست|نیست|بود|باشد|باد|شود|می شود|شده است)(م|ی|یم|ید|ند|\s)", text)
        for regex in regexes:
            s, e = regex.start(), regex.end()
            Q = " ".join(text[:e].split()[:-2]) + " چگونه " + text[s:e].strip() + "؟"
            A = text
            result.append({"Question": Q, "Answer": A})
        return result


    def prepositional_verbs(self, text):
        text = text.replace("\u200c", " ")
        result = []
        regexes = re.finditer(r"\s(از|به|با|در|برای)\s", text)
        for regex in regexes:
            s, e = regex.start(), regex.end()
            # check live and not live subject
            A = text[e:].split()[0]
            live_or_not = self.check_live_noun(A)
            place = self.check_place(A)
            if live_or_not:
                live_vs_not = "چه کسی "
            elif place:
                live_vs_not = "کجا "
            else:
                live_vs_not = "چه چیزی "
            Q = text[:e] + live_vs_not + " ".join(text[e:-1].split()[1:]) + "؟"
            result.append({"Question": Q, "Answer": A})
        return result


    def do_questions(self, text):
        result = []
        regexes = re.finditer(r"\s(زیرا|چون|چرا که|به این دلیل که)\s", text)
        s = -1
        for regex in regexes:
            s, e = regex.start(), regex.end()

        pos_tags = self.tagger.tag(word_tokenize(text))
        text = text.replace("\u200c", " ")
        for word, tag in pos_tags:
            if tag == 'V':
                # check pos or neg
                if word[0] == "ن":
                    A = "خیر."
                    index_of_n = text.find(word)
                    new_text = ""
                    # remove n
                    for i, c in enumerate(text):
                        if i != index_of_n:
                            new_text += c
                    text = new_text
                else:
                    A = "بله."
                # check if there is a describtion after verb
                if text.find(word) < s:
                    Q = "آیا " + text[:s].strip() + "؟"
                    A += text[s:]
                else:
                    Q = "آیا " + text[:-1] + "؟"

                result.append({"Question": Q, "Answer": A})
                break
        return result


    def including_keywords(self, text):
        text = text.replace("\u200c", " ")
        text = text[:-1] if text[-1] is '.' else text
        result = []
        regexes = re.finditer(r"\s(باعث|عامل|دلیل|برهان|موجب)\s", text)
        for regex in regexes:
            s, e = regex.start(), regex.end()
            A = text[:s].strip()
            live_or_not = self.check_live_noun(A)
            if live_or_not:
                live_vs_not = "چه کسی "
            else:
                live_vs_not = "چه چیزی"
            Q = live_vs_not + text[s:] + "؟"
            result.append({"Question": Q, "Answer": A})
        regexes = re.finditer(r"\s(زیرا|چون|چرا که|به این دلیل که)\s", text)
        for regex in regexes:
            s, e = regex.start(), regex.end()
            Q = "چرا " + text[:s] + "؟"
            A = "چون " + text[e:].strip() + '.'
            result.append({"Question": Q, "Answer": A})
        regexes = re.finditer(r"\s(می توان نتیجه گرفت)\s", text)
        for regex in regexes:
            s, e = regex.start(), regex.end()
            Q = text[:s] + " چه نتیجه ای می توان گرفت؟"
            A = text[e:].strip() + '.'
            result.append({"Question": Q, "Answer": A})
        return result
      

    def start_with_pronoun(self, text):
        text = text[:-1] if text[-1] is '.' else text
        pos_tags = self.tagger.tag(word_tokenize(text))
        text = text.replace("\u200c", " ")
        result = []
        regexes = re.finditer(r"^(من|تو|او)\s", text)
        for regex in regexes:
            s, e = regex.start(), regex.end()
            for word, tag in pos_tags:
                if tag == 'V':
                    if self.v_to_thirdperson.get(word) is None:
                      return {}
                    word = word.replace("_", " ")
                    text = text.replace(word, self.v_to_thirdperson[word])
            Q = "چه کسی " + text[e:] + "؟"
            A = text[:e].strip()
            result.append({"Question": Q, "Answer": A})
        regexes = re.finditer(r"^(ما|شما|ایشان|آن ها|آنها)\s", text)
        for regex in regexes:
            s, e = regex.start(), regex.end()
            for word, tag in pos_tags:
                if tag == 'V':
                    if self.v_to_thirdperson.get(word) is None:
                      return {}
                    word = word.replace("_", " ")
                    text = text.replace(word, self.v_to_thirdperson[word])
            Q = "چه کسانی " + text[e:] + "؟"
            A = text[:e]
            result.append({"Question": Q, "Answer": A})
        return result
    
    
    def when_questions(self, text):
        result = []
        if bool(re.search('(زیرا|چون|چرا که|به این دلیل که)', text)):
          return result
        text = text.replace("\u200c", " ")
        text = text[:-1] if text[-1] is '.' else text
        extractor = TimeExtraction()
        extractions = extractor.run(text)
        for t in (extractions['datetime'].keys()):
            s, e = t[1:-1].split(', ')
            s, e = int(s), int(e)
            Q = text[:s] + 'چه زمانی' + text[e:] + '؟'
            A = text[s:e]
            result.append({"Question": Q, "Answer": A})
        return result


    def number_question(self,text):
        text = text.replace("\u200c", " ")
        result = []
        text = text[:-1] if text[-1] is '.' else text 
        normalizer = Normalizer()
        text = normalizer.normalize(text)
        extractor = NumberExtractor()
        numbers = extractor.run(text)
        if numbers:
            for num in numbers:
                Q = text[:num['span'][0]] + 'چند' + text[num['span'][1]:] + '؟'
                A = num['phrase']
                result.append({"Question": Q, "Answer": A})
        return result
    
    def adjective_additive(self, text):
        # text = text[:-1] if text[-1] is '.' else text
        normalizer = Normalizer()
        text = normalizer.normalize(text)
        def check_live(word):
            stemmer = Stemmer()
            for pat in self.live_patterns:
                if stemmer.stem(word) in pat:
                    return True
            return False 

        result = []
        if bool(re.search('(زیرا|چون|چرا که|به این دلیل که)', text)):
            return result
        
        tagger = POSTagger(model='resources/postagger.model')
        word_taggs = tagger.tag(word_tokenize(text))
        words = [w[0] for w in word_taggs]
        taggs = [w[1] for w in word_taggs]

        for i in range(1,len(taggs)):
            if taggs[i-1]=='Ne':
                if taggs[i]=='N' and check_live(words[i]):
                    Q = text.replace(words[i],'چه کسی')
                    Q = Q[:-1] if Q[-1] is '.' else Q
                    Q = Q + '؟'
                    A = words[i-1] + ' ' + words[i]
                    result.append({"Question": Q, "Answer": A})
                elif taggs[i] == 'AJ' or (taggs[i]=='N' and not(check_live(words[i]))):
                    l = words[:i-1]+ ['چه', words[i-1]+' ی']+words[i+1:]
                    Q = " ".join(l)
                    Q = Q[:-1] if Q[-1] is '.' else Q
                    Q = Q.strip() + '؟'
                    A = words[i-1] + ' ' + words[i]
                    result.append({"Question": Q, "Answer": A})
        return result


    def run(self, text):
        result = []
        try:
            # check regexes
            # transitive verbs 
            result += self.transitive_verbs(text)
            # Esnadi verbs
            result += self.esnadi_verbs(text)
            # verbs which have prepositions
            result += self.prepositional_verbs(text)
            # Do, Does Questions
            result += self.do_questions(text)
            # Keywords
            result += self.including_keywords(text)
            # starting with pronoun
            result += self.start_with_pronoun(text)
            # when questions
            result += self.when_questions(text)
            # questions with numbers
            result += self.number_question(text)
            # adjective additive
            result += self.adjective_additive(text)
            return result
        except:
            return result

<div dir=rtl>

* **تابع transitive_verbs:** این تابع برای سوال کردن مفعول جمله بکار می رود، بدین صورت که چون "را" علامت مفعولی است، با استفاده از یافتن  آن می توان این کار را انجام داد. در صورتی که مفعول به یک جاندار اشاره کند سوال با "چه کسی" خواهد بود ولی در غیر این صورت چون لیست کلمات جاندار خیلی کامل نیست، سوال با "چه چیزی/کسی" خواهد بود.
* **تابع esnadi_verbs:** این تابع برای سوال کردن مسند جمله است و با یافتن فعل های اسنادی در جمله، می توان کلمه قبل آن ها را که مسند می باشد را سوال پرسید.
* **تابع prepositional_verbs:** این تابع برای سوال کردن متمم های موجود در جمله است و بدین صورت عمل می کند که با یافتن حروف اضافه بر اساس لیست رجکس های مشخص شده، آن ها را سوالی می کند. قابل توجه است که در صورتی که متمم موجود زنده باشد، با "چه کسی"، در صورتی که مکان باشد با "کجا" و در صورتی که هیچ کدام از آن ها نباشید با "چه چیزی" سوال خواهد کرد.
* **تابع do_questions:** این تابع برای سوال پرسیدن با "آیا" بکار می رود و با تشخیص مثبت یا منفی بودن جمله جواب بله یا خیر می دهد.
* **تابع including_keywords:** این تابع برای تولید پرسش و پاسخ در جملاتی به کار می رود که کلید واژه هایی نظیر زیرا، چون، چراکه، باعث، دلیل، عامل و ... را دارند.
* **تابع start_with_pronoun:** این تابع برای تولید پرسش و پاسخ در جملاتی به کار می رود که با ضمیر آغاز می شوند و سوالاتی با فرمت چه کسی ....؟ تولید می کند.
* **تابع when_questions:** این تابع برای تولید پرسش و پاسخ هایی تولید می شود که در آن ها زمان و تاریخ ذکر شده باشد و سوالاتی با فرمت چه زمانی ... ؟ تولید می کند.
* **تابع number_question:** این تابع برای سوال کردن در مورد اعداد در جمله است و در مورد اینکه آن عدد چند است سوال میپرسد.
* **تابع adjective_additive:** این تابع برای سوال در مورد صفت در ترکیب وصفی یا مضاف الیه در ترکیب اضافی است. اگر ترکیب اضافی باشد و مضاف الیه یک فرد باشد از "چه کسی؟" استفاده میکند در غیر این صورت از "چه + مضاف + ی" برای پرسش استفاده میکند.

</div>

# Results

In [ ]:
str1 = "از این موضوع می توان نتیجه گرفت که اتهام مربوطه به او وارد نیست."
str2 = "حرکت بار الکتریکی باعث ایجاد میدان الکترومغناطیسی در فضا می شود."
str3 = "سرور دانشگاه پایین است چرا که دیروز مشکل سخت افزاری پیش آمد."
str4 = "جاذبه باعث افتادن سیب از درخت می شود."
str5 = "در امتحان موفق شدم زیرا تمرین های زیادی حل کردم."
str6 = "من غذا را می خورم."
str7 = "من از قبولی در دانشگاه دلسرد گشتم."
str8 = "فردا صبح به شیراز سفر می کنم."
str9 = "در امتحان موفق نشدم زیرا تمرین های زیادی حل نکردم."
str10 = "من دنبال او گشتم."
str11 = "من به دانشگاه رفتم و ناهار را خوردم."
str12 = "ما به دانشگاه رفتیم و ناهار را خوردیم."
str13 = "دوستم غذایش را خورد."
str14 = "من نیوشا را دیدم."
str15 = "غذا دیروز من را به بیمارستان کشاند."
str16 = "سوختن کامپیوترم عامل قطعی برق دیروز بود."
str17 = "من فردا به تبریز سفر خواهم کرد."
str18 = "من فردا به تبریز سفر می کنم."
str19 = "من از خانه بیرون رفتم و سارا را دیدم."
str20 = "من سال ۱۳۷۵ شمسی به دنیا آمده ام."

In [ ]:
model  = QAExtraction()
input_text = str1
result = model.run(input_text)
print(json.dumps(result, indent=4, ensure_ascii=False))

[
    {
        "Question": "از این موضوع می توان نتیجه گرفت که اتهام مربوطه به چه کسی وارد نیست؟",
        "Answer": "او"
    },
    {
        "Question": "آیا از این موضوع می توان نتیجه گرفت که اتهام مربوطه به او وارد نیست؟",
        "Answer": "بله."
    },
    {
        "Question": "از این موضوع چه نتیجه ای می توان گرفت؟",
        "Answer": "که اتهام مربوطه به او وارد نیست."
    }
]


In [ ]:
strs = [eval("str" + str(i)) for i in range(1, 21)]
for s in strs:
    print(s)
    input_text = s
    result = model.run(input_text)
    print(json.dumps(result, indent=4, ensure_ascii=False))

# References



*   Persian names dataset from [here](https://www.kaggle.com/datasets/rezaali/dataset-for-names-of-people-in-persian-language)
*   Places dataset from [here](https://github.com/language-ml/parsi.io)
*   Verb roots from [here](https://www.peykaregan.ir/dataset/%D9%85%D8%AC%D9%85%D9%88%D8%B9%D9%87-%D8%A7%D9%81%D8%B9%D8%A7%D9%84-%D8%AA%D8%B5%D8%B1%DB%8C%D9%81%E2%80%8C%D8%B4%D8%AF%D9%87-%D9%81%D8%A7%D8%B1%D8%B3%DB%8C)

